In [ ]:
import pickle
import numpy as np
from qiskit import *
import scipy.linalg as la
from mpl_axes_aligner import shift
from itertools import product
import matplotlib.pyplot as plt
from qulacsvis import circuit_drawer
from qulacs.gate import CNOT, RX, RY, RZ
from qiskit.quantum_info.states.random import random_density_matrix
from qiskit.quantum_info import Statevector, DensityMatrix, partial_trace
plt.rcParams.update(
    {"text.usetex": True, "font.family": "serif", "font.size": 10}
)

In [ ]:
def dictionary_of_actions(num_qubits):
    """
    Returns dictionary of available actions for particular qubit
    """
    dictionary = dict()
    i = 0
    for c, x in product(range(num_qubits),
                        range(1, num_qubits)):
        dictionary[i] =  [c, x, num_qubits, 0]
        i += 1
    for r, h in product(range(num_qubits),
                           range(1, 4)):
        dictionary[i] = [num_qubits, 0, r, h]
        i += 1
    return dictionary

def make_circuit_qiskit(action, qubits):
    """
    Makes quantum circuit
    """
    ctrl = action[0]
    targ = (action[0] + action[1]) % qubits
    rot_qubit = action[2]
    rot_axis = action[3])
    if ctrl < qubits:
        circuit.cx([ctrl], [targ])
    if rot_qubit < qubits:
        if rot_axis == 1:
            circuit.rx(0, rot_qubit)
        elif rot_axis == 2:
            circuit.ry(0, rot_qubit)
        elif rot_axis == 3:
            circuit.rz(0, rot_qubit)
        else:
            stop
    return circuit

def eigenvals_func(theta, rdm, ansatz):
    """
    Return the eigenvalues
    """
    qc_eigenvals = ansatz
    dm_evo = rdm.evolve(qc_eigenvals)
    dm_evo = dm_evo.data
    diagnls = list((np.diagonal(dm_evo)).real)
    diag_A = []
    diag_A.append(diagnls)
    return diag_A, dm_evo, qc_eigenvals

In [ ]:
line_style_list = ['r-', 'k--']
qubits, rank, seed = 2,4, 9
seed_list = range(1, 50+1)

# ---------- LOADTING DATA -------- #
HEA_err_data, agent_err_data, HEA_layer_data, agent_layer_data = [], [], [], []
for seed in seed_list:
    print('Seed', seed)
    if seed != 1000:

        data = np.load(f'results/global_COBYLA/h_s_{qubits}_rank_{rank}_{seed}/summary_100.npy',allow_pickle=True)[()]
        succ_ep_data = []
        with (open(f"plot_data/{qubits}_qubit_mixed_rank_{rank}_seed_{seed}_succ_ep_data.p", "rb")) as openfile:
            while True:
                try:
                    succ_ep_data.append(pickle.load(openfile))
                except EOFError:
                    break

        circuit_data = []
        with (open(f"plot_data/{qubits}_qubit_mixed_rank_{rank}_seed_{seed}_circuit_data.p", "rb")) as openfile:
            while True:
                try:
                    circuit_data.append(pickle.load(openfile))
                except EOFError:
                    break

        HEA_data = []
        with (open(f"plot_data/{qubits}_qubit_mixed_rank_{rank}_seed_{seed}_lHEA_data_tot_state_50.p", "rb")) as openfile:
            while True:
                try:
                    HEA_data.append(pickle.load(openfile))
                except EOFError:
                    break

        HEA_layer_list = [i * 6 for i in HEA_data[0]['layer_list']]
        HEA_layer_data.append(HEA_layer_list)
        HEA_abs_err_list = HEA_data[0]['eig_err_list']
        HEA_err_data.append(HEA_abs_err_list)


        # ----------- USING DATA --------- #
        succ_ep_list, succ_ep_err_list = succ_ep_data[0]['succ_ep_list'], succ_ep_data[0]['succ_ep_err_list']
        depth_list, oneq_gate_list = circuit_data[0]['depth_list'], circuit_data[0]['oneq_gate_list']
        twoq_gate_list, gate_num_list = circuit_data[0]['twoq_gate_list'], circuit_data[0]['gate_num_list']
        succ_ep = succ_ep_list[gate_num_list.index(min(gate_num_list))]
        thetas = list(data['train'][succ_ep]['opt_ang'])[-1]
        state_to_diag_gen = random_density_matrix(2**qubits, rank, seed=seed)
        ture_eig = np.sort(la.eig(state_to_diag_gen)[0].real)
        final_circuit = QuantumCircuit.from_qasm_str(data['train'][succ_ep]['save_circ'][-1])
        inf_eig = np.sort(eigenvals_func(thetas, state_to_diag_gen, final_circuit)[0])[0]

        #------- ABS ERROR APPENDATION -----------#
        gate_num_circ_list, abs_err_list = [], []
        for l in range(np.min(gate_num_list)+1):
            final_circuit = QuantumCircuit.from_qasm_str(data['train'][succ_ep]['save_circ'][-1])
            final_circuit.data = final_circuit[:l]
            inf_eig = np.sort(eigenvals_func(thetas, state_to_diag_gen, final_circuit)[0])[0]
            err = 0
            for eigno in range(qubits**2):
                err += np.abs(inf_eig[eigno] - ture_eig[eigno])**2
            gate_num_circ_list.append(1+l)
            abs_err_list.append(err)
        agent_err_data.append(abs_err_list)
        agent_layer_data.append(gate_num_circ_list)


In [ ]:
#-------- PLOTTING ----------#
HEA_abs_err_list  = []

for er_per_layer in range(len(HEA_err_data[0])):
    err_ave_per_layer = 0
    for er_per_seed in range(len(HEA_err_data)):
        err_ave_per_layer +=HEA_err_data[er_per_seed][er_per_layer]
    HEA_abs_err_list.append( err_ave_per_layer / len(HEA_err_data))

In [ ]:
from itertools import zip_longest
agent_abs_err_std, agent_abs_err_list = [], []
def column_wise_sum(rows):
    columns = zip_longest(*rows, fillvalue=0)
    return list(columns)
for k in column_wise_sum(agent_err_data):
    agent_abs_err_list.append(np.mean(k))
    agent_abs_err_std.append(np.std(k))

In [ ]:
agent_abs_err_std, agent_abs_err_list = np.asarray(agent_abs_err_std), np.asarray(agent_abs_err_list)

In [ ]:
HEA_best_err_list = []
for el in HEA_err_data:
    HEA_best_err_list.append(el[-1])
agent_best_err_list = []
for el in agent_err_data:
    agent_best_err_list.append(el[-1])
HEA_best_indx,agent_best_indx=HEA_best_err_list.index(min(HEA_best_err_list)), agent_best_err_list.index(min(agent_best_err_list))

In [ ]:
fig, ax1 = plt.subplots( 1, 1, sharey=True, figsize = (4, 3.5))
line_style_list = [ 'r-o', 'b-x', 'r--o', 'b--x' ]
ax1.semilogy(range(1, len(agent_abs_err_list)+1), agent_abs_err_list, line_style_list[0], markerfacecolor='none', label = f'RL-agent')
ax1.semilogy(agent_layer_data[agent_best_indx], agent_err_data[agent_best_indx], line_style_list[2], markerfacecolor='none', label = f'RL-agent (best)')
ax1.semilogy(HEA_layer_data[0], HEA_abs_err_list, line_style_list[1], label = f'LHEA')
ax1.semilogy(HEA_layer_data[HEA_best_indx], HEA_err_data[HEA_best_indx], line_style_list[3], label = f'LHEA (best)')

# ax1.set_title(f'qubit {qubits}, rank {rank}')
ax1.set_ylabel('$\sum_i^{m}(\\lambda_i - \\lambda^{\\prime}_i)^2$')
ax1.set_xlabel('Number of gates')
org = 0.2  # Origin
pos = 8  # Position the origins is aligned
expand = True  # Switch the aligning method
shift.xaxis(ax1, pos, org)
ax1.set_xticks(list(range(1,HEA_layer_data[0][-1]+5,4)))
ax1.legend(ncol = 2, bbox_to_anchor=[0.9, 1.24])
ax1.legend(ncol = 2, bbox_to_anchor=[1, 1.25])
ax1.grid(color='k', linestyle='--', linewidth=1.2, alpha = 0.2)
fig.tight_layout()
# fig.savefig(f'plot/{qubits}_qubit_all_rank.pdf')
# fig.savefig(f'plot/{qubits}_qubit_all_rank.png')